In [1]:
import sys
sys.path.append('..')
from MPRA_exp.utils import *
from MPRA_exp.models import Beluga
from MPRA_exp.datasets import SeqLabelDataset

warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

In [2]:
def get_pred(model, test_data_loader, device='cuda'):
    model = model.to(device)
    y_pred = []
    with torch.no_grad():
        model.eval()
        for (x, y) in tqdm(test_data_loader):
            x = x.to(device)
            x_rc = onehots_reverse_complement(x).to(device)
            pred_1 = model(x)
            pred_2 = model(x_rc)
            pred = (pred_1 + pred_2) / 2
            y_pred.extend(pred.cpu().detach().numpy())
    y_pred = np.array(y_pred)
    return y_pred

In [3]:
trained_model_path = '/home/hxcai/cell_type_specific_CRE/MPRA_zero_shot/pretrained_models/beluga/deepsea.beluga.pth'
model = Beluga()
state_dict = torch.load(trained_model_path) 
new_state_dict = {k.replace('module.model.', ''): v for k, v in state_dict.items()}
model.load_state_dict(new_state_dict)

dataset = SeqLabelDataset(seq_exp_path='/home/hxcai/cell_type_specific_CRE/data/SirajMPRA/SirajMPRA_total.csv',
                          input_column='seq', seq_pad_len=2000)
test_data_loader = DataLoader(dataset, batch_size=1024, shuffle=False, num_workers=8)

y_pred = get_pred(model, test_data_loader)
np.save(f'/home/hxcai/cell_type_specific_CRE/MPRA_zero_shot/pretrained_models/Beluga_Siraj_pred.npy', y_pred)

not assigned labels, initial with zeros


100%|██████████| 572/572 [03:07<00:00,  3.05it/s]
